In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.

In [ ]:
from tsdm.datasets import KIWI_RUNS
from tsdm.encoders import *

In [ ]:
dataset = KIWI_RUNS()
ts = dataset.timeseries.astype(float)

In [ ]:
column_encoders = {}
for col in ts:
    match dataset.value_features.scale[col]:
        case "percent":
            column_encoders[col] = LogitBoxCoxEncoder() @ MinMaxScaler(xmin=0, xmax=100)
        case "linear":
            column_encoders[col] = IdentityEncoder()
        case "absolute":
            column_encoders[col] = BoxCoxEncoder()

column_encoders

# Encoder Demo

In [ ]:
enc = Standardizer() @ FrameEncoder(
    column_encoders,
    index_encoders={
        "run_id": IdentityEncoder(),
        "experiment_id": IdentityEncoder(),
        "measurement_time": MinMaxScaler() @ TimeDeltaEncoder(),
    },
)
enc.fit(ts)

In [ ]:
%matplotlib inline
ts.hist(figsize=(16, 6), layout=(3, 5), bins=20);

In [ ]:
%matplotlib inline
encoded = enc.encode(ts)
encoded.hist(figsize=(16, 6), layout=(3, 5), bins=20);

In [ ]:
decoded = enc.decode(encoded)
((decoded - ts) ** 2).mean()

In [ ]:
decoded

## Encoder Complete

In [ ]:
enc = (
    Frame2Tensor(
        groups={
            "i": ["run_id", "experiment_id"],
            "t": ["measurement_time"],
            "x": ...,
        }
    )
    @ Standardizer()
    @ FrameEncoder(
        column_encoders,
        index_encoders={
            "run_id": IdentityEncoder(),
            "experiment_id": IdentityEncoder(),
            "measurement_time": MinMaxScaler() @ TimeDeltaEncoder(),
        },
    )
)
enc.fit(ts)

In [ ]:
encoded = enc.encode(ts)

In [ ]:
decoded = enc.decode(encoded)